In [1]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import pandas as pd
from langchain.chains import LLMChain
from langfuse.callback import CallbackHandler
from langchain.llms import OpenAI
import os
from langchain.chains import SequentialChain


load_dotenv()

handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [18]:
MODEL = "gpt-4-1106-preview"

OUTPUT_FOLDER = "guis"
DATASET = "dataset.csv"

df = pd.read_csv(DATASET, sep=";")
df

,ID,UI Number,Summary
0,1,39054,"A page from a marketplace app, presenting a fo..."
1,2,15711,"A page from a music player app, presenting pla..."
2,3,494,"A page from a music app, presenting options to..."
3,4,69587,"A page from an email management app, presentin..."
4,5,377,"A page from a travel booking app, presenting o..."
5,6,56735,A page from a restaurant review and discovery ...
6,7,491,"A page from a book reading app, presenting a s..."
7,8,34444,"A page from a real estate app, presenting list..."
8,9,3261,"A page from a real estate app, presenting opti..."
9,10,49629,"A page from a messaging app, presenting a list..."


In [3]:
from prompts import SUMMARY2REQUIREMENTS_ZS, REQUIREMENTS2IMPLEMENTATION_ZS, IMPLEMETATION2STRUCTURE_ZS, STRUCTURE2CODE_ZS

def s2r2e2s2c (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIREMENTS_ZS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=REQUIREMENTS2IMPLEMENTATION_ZS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_ZS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_ZS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))


In [4]:
from prompts import SUMMARY2REQUIRMENTS_FS, REQUIREMENTS2IMPLEMENTATION_FS, IMPLEMETATION2STRUCTURE_FS, STRUCTURE2CODE_FS

def s2r2e2s2c_fs (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIRMENTS_FS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=REQUIREMENTS2IMPLEMENTATION_FS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_FS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_FS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))

In [5]:
from prompts import SUMMARY2STRUCTURE_ZS, STRUCTURE2CODE_ZS

def s2s2c (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2STRUCTURE_ZS, output_key="structure",)
    chain_2 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_ZS, output_key="code",)

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))

In [6]:
from prompts import SUMMARY2STRUCTURE_FS, STRUCTURE2CODE_FS

def s2s2c_fs (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2STRUCTURE_FS, output_key="structure",)
    chain_2 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_FS, output_key="code",)

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))

In [7]:
from prompts import SUMMARY2CODE_ZS

def s2c(summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=SUMMARY2CODE_ZS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [8]:
from prompts import CoT_ZS

def s2c_xot_zs(summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=CoT_ZS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [9]:
from prompts import CoT_FS

def s2c_xot_fs(summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=CoT_FS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [10]:
import re

def extract_html(code, file, folder):
    html_content = re.search(r'<!DOCTYPE html>.*</html>', code, re.DOTALL)
    extracted_text = html_content.group()
    try:
        with open(f"{folder}/{file}.html", 'w') as f:
            f.write(extracted_text)
    except Exception as e:
        print(f"{file} failed")
        print(e)

In [11]:
def prototyping(summary,folder):
   
    code = s2c(summary)
    extract_html(code,"instruction",folder)    
    
    code = s2c_xot_zs(summary)
    extract_html(code,"CoT_zs",folder)    
    
    code = s2c_xot_fs(summary)
    extract_html(code,"CoT_fs",folder)    

    code = s2s2c(summary)
    extract_html(code,"simple_pd_zs",folder)    

    code = s2s2c_fs(summary)
    extract_html(code,"simple_pd_fs",folder)    

    code = s2r2e2s2c(summary)
    extract_html(code,"complex_pd_zs",folder)    

    code = s2r2e2s2c_fs(summary)
    extract_html(code,"complex_pd_fs",folder)    
   


In [19]:
def pipeline():
    for index, row in df.iterrows():
        os.makedirs(f"{OUTPUT_FOLDER}/{row['UI Number']}")
        prototyping(row['Summary'],f"{OUTPUT_FOLDER}/{row['UI Number']}")


In [20]:
pipeline()

guis/39054
guis/15711
guis/494
guis/69587
guis/377
guis/56735
guis/491
guis/34444
guis/3261
guis/49629
guis/20163
guis/5983
guis/61446
guis/56055
guis/38961
guis/1264
guis/46038
guis/38460
guis/33576
guis/30740
guis/42444
guis/17084
guis/7757
guis/34676
guis/3513
guis/19344
guis/53054
guis/43802
guis/18107
guis/27546
